In [1]:
from keras.utils import to_categorical
from keras.layers import Dropout
from keras.layers import Dense
from keras import models
from keras import optimizers
from keras import backend as K
import pandas as pd
import tensorflow as tf
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import math
import gensim
from gensim.models import Word2Vec 
import time

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Prepare the data

In [2]:
def prepare(df):

    y = df.label.tolist()
    X = np.matrix(df.drop(labels = ['label'], axis = 1)).astype(np.float)
    print(X.shape)
    
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    
    return X, y

# ANN Metrics

In [3]:
def recall(y_true, y_pred):
   
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
    
def precision(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    
    myPrecision = precision(y_true, y_pred)
    myRecall = recall(y_true, y_pred)
    return 2*((myPrecision*myRecall)/(myPrecision+myRecall+K.epsilon()))

# ANN Model

In [4]:
trainPath = [#'/scratch/by8jj/stratified samples/test of test/train/train_11.csv',
             #'/scratch/by8jj/stratified samples/test of test/train/train_12.csv',
             #'/scratch/by8jj/stratified samples/test of test/train/train_13.csv',
             #'/scratch/by8jj/stratified samples/test of test/train/train_14.csv',
             #'/scratch/by8jj/stratified samples/test of test/train/train_15.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_16.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_17.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_18.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_19.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_20.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_21.csv',
            ]

In [5]:
testPath = [#'/scratch/by8jj/stratified samples/test of test/test/test_12.csv',
           #'/scratch/by8jj/stratified samples/test of test/test/test_13.csv',
           #'/scratch/by8jj/stratified samples/test of test/test/test_14.csv',
           #'/scratch/by8jj/stratified samples/test of test/test/test_15.csv',
           #'/scratch/by8jj/stratified samples/test of test/test/test_16.csv',
            '/scratch/by8jj/stratified samples/test of test/test/test_17.csv',
           '/scratch/by8jj/stratified samples/test of test/test/test_18.csv',
           '/scratch/by8jj/stratified samples/test of test/test/test_19.csv',
           '/scratch/by8jj/stratified samples/test of test/test/test_20.csv',
            '/scratch/by8jj/stratified samples/test of test/test/test_21.csv',
           '/scratch/by8jj/stratified samples/test of test/test/test_22.csv',
            ]

In [6]:
with tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=16)) as sess:
    K.set_session(sess)
    for train, test in zip(trainPath, testPath):

        #drop different port numbers
        df_train = pd.read_csv(train)
        temp1 = set(df_train.columns)

        #for test in testPath:
        df_test = pd.read_csv(test)
        temp2 = set(df_test.columns)

        df_train = df_train.drop(list(temp1 - temp2), axis = 1)
        df_test = df_test.drop(list(temp2 - temp1), axis = 1)


        X, y = prepare(df_train)
        dim = X.shape[1]
        print('dim is %s'%dim)

        model = models.Sequential()
        model.add(Dense(dim//2, input_dim = dim, kernel_initializer='uniform', activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
        adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0000002, amsgrad=False)
        #sgd = optimizers.sgd(lr=0.0005, momentum=0.5, decay=0.000002, nesterov=False)
        model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy',f1,recall,precision])


        result = model.fit(X, y, epochs=25, batch_size=256, validation_split=0.3) 



        X_test, y_test = prepare(df_test)
        y_pred = model.predict(X_test).tolist()
        
        
        '''
        temp = [1 if x[1]>0.8 else 0 for x in y_pred]
        cm= confusion_matrix(y_test, temp)
        tn, fp, fn, tp = cm.ravel()
        precision=tp/(tp+fp)
        recall=tp/(tp+fn)
        fpr = fp/(fp+ tn)
        accuracy = (tp + tn)/(tn + tp + fn + fp)
        F1 = 2 * (precision * recall) / (precision + recall)
        print("precision:", precision*100)
        print("recall:", recall*100)
        print("false positive rate:", fpr*100)
        print("accuracy",  accuracy*100)
        print("F1-score", F1)
        '''
        
        pd.DataFrame([round(x[0], 6) for x in y_pred]).to_csv('/scratch/by8jj/stratified samples/test of test/ann/' + train[-12:-4] + '-' + test[-11:-4] + '.csv', index = False)


(3292475, 73)
dim is 73
Train on 2304732 samples, validate on 987743 samples
Epoch 1/25
2304732/2304732 [==============================] - 17s 8us/step - loss: 0.1796 - acc: 0.9395 - f1: 0.9332 - recall: 0.9823 - precision: 0.8896 - val_loss: 0.1253 - val_acc: 0.9521 - val_f1: 0.9465 - val_recall: 0.9893 - val_precision: 0.9077
Epoch 2/25
2304732/2304732 [==============================] - 17s 7us/step - loss: 0.1291 - acc: 0.9523 - f1: 0.9466 - recall: 0.9902 - precision: 0.9071 - val_loss: 0.1199 - val_acc: 0.9529 - val_f1: 0.9474 - val_recall: 0.9919 - val_precision: 0.9071
Epoch 3/25
2304732/2304732 [==============================] - 17s 8us/step - loss: 0.1247 - acc: 0.9529 - f1: 0.9473 - recall: 0.9912 - precision: 0.9076 - val_loss: 0.1174 - val_acc: 0.9535 - val_f1: 0.9481 - val_recall: 0.9921 - val_precision: 0.9082
Epoch 4/25
2304732/2304732 [==============================] - 18s 8us/step - loss: 0.1221 - acc: 0.9534 - f1: 0.9478 - recall: 0.9913 - precision: 0.9084 - val_loss

Epoch 12/25
2581560/2581560 [==============================] - 20s 8us/step - loss: 0.1174 - acc: 0.9522 - f1: 0.9459 - recall: 0.9850 - precision: 0.9101 - val_loss: 0.1122 - val_acc: 0.9533 - val_f1: 0.9474 - val_recall: 0.9924 - val_precision: 0.9068
Epoch 13/25
2581560/2581560 [==============================] - 20s 8us/step - loss: 0.1172 - acc: 0.9522 - f1: 0.9459 - recall: 0.9849 - precision: 0.9103 - val_loss: 0.1118 - val_acc: 0.9531 - val_f1: 0.9473 - val_recall: 0.9923 - val_precision: 0.9065
Epoch 14/25
2581560/2581560 [==============================] - 21s 8us/step - loss: 0.1168 - acc: 0.9522 - f1: 0.9459 - recall: 0.9848 - precision: 0.9104 - val_loss: 0.1114 - val_acc: 0.9531 - val_f1: 0.9473 - val_recall: 0.9924 - val_precision: 0.9065
Epoch 15/25
2581560/2581560 [==============================] - 21s 8us/step - loss: 0.1166 - acc: 0.9525 - f1: 0.9461 - recall: 0.9847 - precision: 0.9109 - val_loss: 0.1113 - val_acc: 0.9534 - val_f1: 0.9475 - val_recall: 0.9923 - val_pr

Epoch 19/25
2849821/2849821 [==============================] - 22s 8us/step - loss: 0.1316 - acc: 0.9550 - f1: 0.9554 - recall: 0.9930 - precision: 0.9209 - val_loss: 0.1273 - val_acc: 0.9556 - val_f1: 0.9560 - val_recall: 0.9936 - val_precision: 0.9214
Epoch 20/25
2849821/2849821 [==============================] - 22s 8us/step - loss: 0.1313 - acc: 0.9551 - f1: 0.9555 - recall: 0.9929 - precision: 0.9211 - val_loss: 0.1270 - val_acc: 0.9556 - val_f1: 0.9560 - val_recall: 0.9937 - val_precision: 0.9214
Epoch 21/25
2849821/2849821 [==============================] - 22s 8us/step - loss: 0.1311 - acc: 0.9551 - f1: 0.9555 - recall: 0.9930 - precision: 0.9210 - val_loss: 0.1268 - val_acc: 0.9557 - val_f1: 0.9561 - val_recall: 0.9937 - val_precision: 0.9215
Epoch 22/25
2849821/2849821 [==============================] - 22s 8us/step - loss: 0.1310 - acc: 0.9551 - f1: 0.9555 - recall: 0.9930 - precision: 0.9210 - val_loss: 0.1268 - val_acc: 0.9557 - val_f1: 0.9561 - val_recall: 0.9937 - val_pr

(1247894, 72)
(2641796, 73)
dim is 73
Train on 1849257 samples, validate on 792539 samples
Epoch 1/25
1849257/1849257 [==============================] - 16s 9us/step - loss: 0.1833 - acc: 0.9333 - f1: 0.9415 - recall: 0.9755 - precision: 0.9106 - val_loss: 0.1264 - val_acc: 0.9464 - val_f1: 0.9525 - val_recall: 0.9802 - val_precision: 0.9267
Epoch 2/25
1849257/1849257 [==============================] - 16s 9us/step - loss: 0.1288 - acc: 0.9471 - f1: 0.9531 - recall: 0.9809 - precision: 0.9271 - val_loss: 0.1227 - val_acc: 0.9471 - val_f1: 0.9532 - val_recall: 0.9817 - val_precision: 0.9265
Epoch 3/25
1849257/1849257 [==============================] - 16s 9us/step - loss: 0.1261 - acc: 0.9477 - f1: 0.9537 - recall: 0.9818 - precision: 0.9274 - val_loss: 0.1214 - val_acc: 0.9483 - val_f1: 0.9542 - val_recall: 0.9812 - val_precision: 0.9289
Epoch 4/25
1849257/1849257 [==============================] - 16s 9us/step - loss: 0.1248 - acc: 0.9480 - f1: 0.9539 - recall: 0.9817 - precision: 0.9

Epoch 8/25
2023210/2023210 [==============================] - 16s 8us/step - loss: 0.1302 - acc: 0.9477 - f1: 0.9461 - recall: 0.9789 - precision: 0.9158 - val_loss: 0.1233 - val_acc: 0.9491 - val_f1: 0.9478 - val_recall: 0.9853 - val_precision: 0.9134
Epoch 9/25
2023210/2023210 [==============================] - 17s 8us/step - loss: 0.1295 - acc: 0.9478 - f1: 0.9462 - recall: 0.9790 - precision: 0.9159 - val_loss: 0.1228 - val_acc: 0.9490 - val_f1: 0.9476 - val_recall: 0.9847 - val_precision: 0.9136
Epoch 10/25
2023210/2023210 [==============================] - 17s 8us/step - loss: 0.1288 - acc: 0.9479 - f1: 0.9463 - recall: 0.9792 - precision: 0.9159 - val_loss: 0.1224 - val_acc: 0.9491 - val_f1: 0.9477 - val_recall: 0.9851 - val_precision: 0.9135
Epoch 11/25
2023210/2023210 [==============================] - 17s 8us/step - loss: 0.1286 - acc: 0.9480 - f1: 0.9464 - recall: 0.9792 - precision: 0.9162 - val_loss: 0.1219 - val_acc: 0.9491 - val_f1: 0.9477 - val_recall: 0.9846 - val_prec